In [9]:
# Continue with regular imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random

import cv2 as cv
import re
import glob
import os

from PIL import Image 
import cv2 as cv
import shutil

cudnn.benchmark = True
plt.ion()   # interactive mode

In [10]:

def partition_files(file_path):
    all_images = glob.glob(file_path)

    # now get the first section of each filename which indicates age 
    # add all of the first things to a list to store a list of all the distinct ages 
    # get the base name of the file so that the folder containing images is not in the fucking way
    dir_names = []
    based_paths = []
    
    for path_name in all_images:
        # change path name to the base name, which will just be the image file
        path_name = os.path.basename(path_name)
        based_paths.append(path_name)
        # partition to get the name divvied up
        partition = re.split('_+', path_name)
        dir_name = partition[0]
        # only add each age one time plsss
        if dir_name not in dir_names:
            dir_names.append(dir_name)
    return dir_names, based_paths

directory_names, based_paths = partition_files("./preprocessed/*.jpg") 

In [11]:
# function to partition asian face files 
import random
def partition_asian_faces(file_path):
    
    based_paths = []
    dir_names = []
    
    all_images = glob.glob(file_path)
    for path_name in all_images:
        
        path_name = os.path.basename(path_name)
        based_paths.append(path_name)
        # split between A and .jpg 
        st = path_name.index("A")
        end = path_name.index(".", st + 1)

        partition = path_name[st +1:end]
        if partition[0] == "0":
            partition = partition[1]
            
        dir_name = partition    
        if dir_name not in dir_names:
            dir_names.append(dir_name)
            
    return dir_names, based_paths
        
asian_directories, asian_based = partition_asian_faces("./asian_faces/*.jpg")  

In [12]:

# now that we have a distinct list of the ages we can create a directory from them
# each directory will just be named the age we want 

def populate_dirs(directory_names, based_paths, raw_path = "./preprocessed/*.jpg", part_path = "./partitioned/"):
    # will need this for reading images
    all_paths = glob.glob(raw_path)
    
    # create the main partitioned directory if it is nonexistent
    
    if os.path.exists(part_path) == False:
        os.mkdir(part_path)
    else:
        # delete it 
        shutil.rmtree(part_path)
        os.mkdir(part_path)
    
    # create the sub directories if they are not exisitent 
    for name in directory_names:
        path  = part_path + "{i}".format(i = name) #eg. ./partitioned/34/
        if os.path.exists(path) == False:
            os.mkdir(path)
            
    # for each item in the list of image paths, create a dir with that item as the title 
    # if the item match the first digits of the file name then add that file to the dir 
    for index, path in enumerate(based_paths):
        # need the first part of the name 
        partition = re.split("_+", path)
        match_case = partition[0] #eg. 34
        # now we have broken down each image path into just the age 
        # if the image path == directory name 
            # add that image path to that directory 
        if match_case in directory_names: # eg. if 34 in [34]
            # open the image 
            current_im = cv.imread(all_paths[index]) # open raw image file and store
            # save the image in the correct new directory 
            final_path  = part_path + "{i}".format(i = match_case) + "/"
            cv.imwrite(final_path + "{i}".format(i = based_paths[index]), current_im)
            
populate_dirs(directory_names, based_paths)

In [13]:
# function to partition asian face files 
def partition_asian_faces(file_path):
    
    based_paths = []
    dir_names = []
    
    all_images = glob.glob(file_path)
    for path_name in all_images:
        
        path_name = os.path.basename(path_name)
        based_paths.append(path_name)
        # split between A and .jpg 
        st = path_name.index("A")
        end = path_name.index(".", st + 1)

        partition = path_name[st +1:end]
        if partition[0] == "0":
            partition = partition[1]
            
        dir_name = partition    
        if dir_name not in dir_names:
            dir_names.append(dir_name)
            
    return dir_names, based_paths
        
asian_directories, asian_based = partition_asian_faces("./asian_faces/*.jpg")  


# add the asian base files to the already populated directories 
def populate_asian_faces(asian_based ,asian_directories, raw_asian_path = "./asian_faces/*.jpg", part_path = "./partitioned/"):
    
    all_paths = glob.glob(raw_asian_path)
    #create the sub directories if they are not exisitent 
    for name in asian_directories:
        path  = part_path + "{i}".format(i = name) #eg. ./partitioned/34/
        if os.path.exists(path) == False:
            os.mkdir(path)
            
    # create a path for the asian faces, each 
    for index, elem in enumerate(asian_based):
        st = elem.index("A")
        end = elem.index(".", st + 1)

        partition = elem[st +1:end] 
        
        if partition in asian_directories:
            current_image  = cv.imread(all_paths[index])
            final_path  = part_path + "{i}".format(i = partition) + "/"
            # cv.imwrite(final_path + "{i}".format(i = elem), current_image)
            cv.imwrite(final_path + "{i}".format(i = elem), current_image)
            

populate_asian_faces(asian_based, asian_directories)       

In [14]:
import splitfolders

prep_path = "./prepared_data/"
    
if os.path.exists(prep_path) == False:
    os.mkdir(prep_path)
else:
    # delete it 
    shutil.rmtree(prep_path)
    os.mkdir(prep_path)
# now we can divide up the partitioned folders/ labels into train val and test 
splitfolders.ratio("./partitioned/", output="prepared_data",
    seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

Copying files: 35376 files [00:45, 780.82 files/s] 


In [15]:
def remove_empty_folders(path_abs):
    walk = list(os.walk(path_abs))
    for path, _, _ in walk[::-1]:
        if len(os.listdir(path)) < 5:
            shutil.rmtree(path)
dir_names = ["train", "test", "val"]        
for dirc in dir_names:
    remove_empty_folders("./prepared_data/{i}/".format(i = dirc))